In [7]:
import os
train_dir = "/home/minimario/Documents/bilevel/new/data/omniglot_resized/images_background"
eval_dir = "/home/minimario/Documents/bilevel/new/data/omniglot_resized/images_evaluation"

In [8]:
from glob import glob
g = glob(f"{train_dir}/*")
alphabets = []
for alphabet in g:
    alphabets.append(alphabet)

In [50]:
from PIL import Image
import numpy as np

def load_image(infilename) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

def save_image( npdata, outfilename ) :
    img = Image.fromarray( np.asarray( np.clip(npdata,0,255), dtype="uint8"), "L" )
    img.save( outfilename )

In [148]:
import random
import numpy as np
def sample_batch(alphabet_index, ways, shots):
    alphabet_dir = alphabets[alphabet_index]
    g = glob(f"{alphabet_dir}/*")
    num_characters = len(g)
    indices = np.random.choice(num_characters, ways, replace=False)

    train_xs = np.zeros((ways*shots, 1, 28, 28))    
    train_ys = np.zeros((ways*shots), dtype=np.int)

    ct = 0
    for i in range(len(indices)):
        index = indices[i]
        path = g[index]
        s = random.sample(os.listdir(path), 2*shots)
        for j in range(shots):
            image = f"{path}/{s[j]}"
            img = load_image(image)
            train_xs[ct][0] = img
            train_ys[ct] = index
            ct += 1

    return torch.tensor(train_xs, dtype=torch.float32), torch.tensor(train_ys)

In [143]:
import torch
import learn2learn as l2l
from torch import nn

def get_alphabet_length(i):
    return len(glob(f"{alphabets[i]}/*"))

class Lambda(nn.Module):

    def __init__(self, fn):
        super(Lambda, self).__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x)

ways = 3
shots = 5
features = l2l.vision.models.ConvBase(output_size=64, channels=1, max_pool=True)
features = torch.nn.Sequential(features, Lambda(lambda x: x.view(-1, 64)))

heads = [torch.nn.Linear(64, get_alphabet_length(i)) for i in range(len(alphabets))]
feature_optimizers = torch.optim.Adam(list(features.parameters()))
head_optimizers = [torch.optim.Adam(list(heads[i].parameters())) for i in range(len(alphabets))]

# head.to(device)


In [167]:
def simplex_projection(s):
    """Projection onto the unit simplex."""
    if np.sum(s) <=1 and np.alltrue(s >= 0):
        return s
    # Code taken from https://gist.github.com/daien/1272551
    # get the array of cumulative sums of a sorted (decreasing) copy of v
    u = np.sort(s)[::-1]
    cssv = np.cumsum(u)
    # get the number of > 0 components of the optimal solution
    rho = np.nonzero(u * np.arange(1, len(u)+1) > (cssv - 1))[0][-1]
    # compute the Lagrange multiplier associated to the simplex constraint
    theta = (cssv[rho] - 1) / (rho + 1.0)
    # compute the projection by thresholding v using theta
    return np.maximum(s-theta, 0)

[0.25 0.75]


In [175]:
n_iterations = 10000
n_inner = 10
ways = 3
shots = 5
loss = nn.CrossEntropyLoss(reduction='mean')

lambdas = 1/30 * np.ones(30)
gamma = 0.01
for it_outer in range(n_iterations):
    # sample some # of alphabets
    alphabet_indices = np.random.choice(len(alphabets), 5, replace=False)
    # alphabet_indices = np.random.choice(5, 5, replace=False)

    # inner loop
    # print("inner loop started")
    for it_inner in range(n_inner):
        a = alphabet_indices[it_inner%5]
        xs, ys = sample_batch(a, ways, shots)
        outer_loop = features(xs)
        inner_loop = heads[a](outer_loop) # [15 (ways * shots), length of alphabet]
        total_loss = loss(inner_loop, ys)
        total_loss.backward()
        head_optimizers[a].step()
    # print("inner loop done")

    # outer update
    # print("outer loop started")
    a = alphabet_indices[it_inner%5]
    total_loss = 0
    lambda_update = np.zeros(30,)
    for a in alphabet_indices:
        xs, ys = sample_batch(a, ways, shots)
        outer_loop = features(xs)
        inner_loop = heads[a](outer_loop) # [15 (ways * shots), length of alphabet]
        inner_loss = loss(inner_loop, ys)
        total_loss += lambdas[a] * inner_loss
        lambda_update[a] += inner_loss
    total_loss.backward()
    print(total_loss)
    feature_optimizers.step()
    lambdas = simplex_projection(lambdas + gamma * lambda_update)
    # print("outer loop done")


/home/minimario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.
tensor(0.5218, grad_fn=<AddBackward0>)
tensor(0.5910, grad_fn=<AddBackward0>)
tensor(0.4812, grad_fn=<AddBackward0>)
tensor(0.2751, grad_fn=<AddBackward0>)
tensor(0.5118, grad_fn=<AddBackward0>)
tensor(0.6790, grad_fn=<AddBackward0>)
tensor(0.2421, grad_fn=<AddBackward0>)
tensor(0.7569, grad_fn=<AddBackward0>)
tensor(0.5509, grad_fn=<AddBackward0>)
tensor(0.6437, grad_fn=

KeyboardInterrupt: 

In [7]:
xs, ys = sample_batch(10, 5)

/home/minimario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.
